# Tests for maps

# Author JJGC

## ilustration of tests for maps functions

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import warnings
import datetime 
print("Last updated on ", time.asctime())

Last updated on  Sat Feb  9 13:53:06 2019


### Notebook configuration

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import tables            as tb
import random
import glob
import warnings
sns.set()

In [4]:
from krcal.core.core_functions      import timeit

In [5]:
from krcal.core.kr_types                 import KrEvent
from krcal.core.analysis_functions       import kr_event
from typing      import List, Tuple, Sequence, Iterable, Dict
from   pandas.core.frame import DataFrame

In [6]:
import logging
log = logging.getLogger()

In [195]:
from krcal.core.io_functions                 import filenames_from_list
from krcal.core.kr_types                     import KrFileName
from  invisible_cities.io.dst_io             import load_dsts
from krcal.core.core_functions               import time_delta_from_time
from krcal.core.analysis_functions           import kr_ranges_and_bins
from krcal.core.analysis_functions           import select_xy_sectors
from krcal.core.analysis_functions           import select_xy_sectors_df
from krcal.core.analysis_functions           import event_map
from   invisible_cities.core.core_functions  import in_range

In [8]:
import cProfile

## Select xy sectors

In [155]:
def select_xy_sectors_df(dst        : DataFrame,
                         bins_x     : np.array,
                         bins_y     : np.array)-> Dict[int, List[DataFrame]]:
    """
    Return a dict of selections organized by xy sector

    Parameters
    ----------
        dst:
        The input data frame.
        bins_x:
        An array of bins along x.
        bins_y:
        An array of bins along y.

    Returns
    -------
        A map of selections defined as Dict[int, List[DataFrame]]
        where for each x (the key in the dict) one has a list
        (corresponding to y cells) of DataFrame (the events selected)

    """
    RGES = {}
    nbins_x = len(bins_x) -1
    nbins_y = len(bins_y) -1
    for i in range(nbins_x):
        dstx = dst[in_range(dst.X, *bins_x[i: i+2])]
        RGES[i] = [dstx[in_range(dstx.Y, *bins_y[j: j+2])] for j in range(nbins_y) ]

    return RGES

In [196]:
def x_and_y_ranges(data, xb, yb, nbx, nby):
    r = True
    for i in range(nbx):
        dstx = data[in_range(data.X, *xb[i: i+2])]
        r & in_range(dstx.X.values, xb[i: i+2][0], xb[i: i+2][1]).all()
        for j in range(nby):
            dsty = dstx[in_range(dstx.Y, *yb[j: j+2])]
            r & in_range(dsty.Y.values, yb[j: j+2][0], yb[j: j+2][1]).all()
    return r

In [197]:
def nmap(RGES):
    for i, ldst in RGES.items():
        DLEN[i] =[len(dst) for dst in ldst]
    return pd.DataFrame.from_dict(DLEN)


In [198]:
def data_frames_are_identical(df1, df2):
    df = df1 == df2 # the resulting df is a df of bools.
    
    # first all() gives a bool per column, creating a Series, seond all() gives a bool for the Series
    return df.eq(True).all().all() 

## Prepare data

In [199]:
x = np.random.random(20) * 100
y = np.random.random(20) * 100
D = {}
D['X'] = x
D['Y'] = y
data = pd.DataFrame.from_dict(D)
print(data)

            X          Y
0   55.914257  91.492164
1   60.688878   9.388756
2   10.691118  87.774070
3   32.305945   8.373585
4   41.942019  44.308992
5   92.218698  32.553244
6   41.502697  62.787282
7   13.337421  73.843563
8   99.247541  37.107046
9   43.818671  50.486132
10  88.302929  78.562866
11  72.665090  84.361013
12  67.921614   5.744892
13  15.386671  32.529157
14  67.510466  31.819258
15  72.848560  99.712277
16   7.163769  46.753639
17  94.106824  98.864060
18  67.512756  27.969804
19  65.544423  64.272719


### Define bins

In [200]:
xb = np.arange(0,101,25)
yb = np.arange(0,101,25)
nbx = len(xb) -1
nby = len(yb) -1
print(f'xb, nbx = {xb, nbx}')
print(f'yb, nby = {yb, nby}')

xb, nbx = (array([  0,  25,  50,  75, 100]), 4)
yb, nby = (array([  0,  25,  50,  75, 100]), 4)


### Check ranges

In [201]:
x_and_y_ranges(data, xb, yb, nbx, nby)

True

### Fill dict

In [202]:
selDict = {}
for i in range(nbx):
    dstx = data[in_range(data.X, *xb[i: i+2])]
    selDict[i] = [dstx[in_range(dstx.Y, *yb[j: j+2])] for j in range(nby) ]

sel = nmap(selDict)

In [203]:
sel

,0,1,2,3
0,0,1,2,0
1,2,1,2,2
2,1,2,1,0
3,1,0,3,2


### Call function and make counting map

In [204]:
selMap = select_xy_sectors_df(data, xb, yb)

In [205]:
sel2 = nmap(selMap)

### Compare data frames

In [206]:
data_frames_are_identical(sel, sel2)

True

## END